# Inference of a recursive probability distribution

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import (SVI, TraceGraph_ELBO)
from pyro.infer.autoguide import AutoNormal

## Example: Geometric distribution

The geometric distribution defined recursively with bernoulli random trials

See documentation example:
- https://pyro.ai/examples/intro_part_i.html
- https://pyro.ai/examples/air.html

In [2]:
def geometric(p, t=None):
    """Counts the number of failures until the first success
    in a sequence of Bernoulli trials.
    """
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)


In [32]:
# Generate some samples.
epsilon = 0.5
data = []
for _ in range(10):
    x = geometric(epsilon)
    data.append(x)
    print('sampled {}'.format(x))
data = torch.tensor(data)

sampled 0
sampled 1
sampled 1
sampled 1
sampled 0
sampled 1
sampled 0
sampled 0
sampled 0
sampled 1


In [33]:
data

tensor([0, 1, 1, 1, 0, 1, 0, 0, 0, 1])

## Guide function

In [24]:
pyro.sample("x", pyro.distributions.Dirichlet())

tensor([0.8809, 0.1191])

In [29]:
def geometric_guide(p):
    #p = pyro.sample("p", dist.Dirichlet(0.5 * torch.ones((2, ))))
    k = pyro.sample("k", dist.Geometric(p))
    return {"k": k}

## Optimizer

In [30]:
optim = pyro.optim.Adam({'lr': 0.1, 'betas': [0.8, 0.99]})

In [31]:
svi = SVI(geometric,
          geometric_guide,
          Adam({'lr': 1e-4}),
          loss=TraceGraph_ELBO())

for i in range(5):
    loss = svi.step(data)
    print('i={}, elbo={:.2f}'.format(i, loss / data.size(0)))

ValueError: Expected parameter probs (Tensor of shape (10,)) of distribution Geometric(probs: torch.Size([10])) to be positive but found invalid values:
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
optim.A